In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pymongo
from pprint import pprint

In [2]:
client = pymongo.MongoClient()
db = client.metrics

In [3]:
def group_by(df, bycols, agg_map):
    """

    @param df:      DataFrame
    @param bycols:  str or list
                        Column(s) to group by
    @param agg_map: dictionary or list of 2-tuples
                        Mapping from column to aggregate function e.g. [("city", "count"), ("salary", "mean"]
    @return:        DataFrame
                        Flattened dataframe, with multi-level index removed
    """
    grps = []
    if type(bycols) == str:
        bycols = [bycols]

    if type(agg_map) == dict:
        agg_map = agg_map.items()

    for k,v in agg_map:
        grp = df[bycols + [k]].groupby(bycols, ).agg(v)
        grp.reset_index(inplace=True)
        grp["%s(%s)" % (v,k)] = grp[k]
        del grp[k]
        grps.append(grp)

    m = grps[0]
    for grp in grps[1:]:
        m = pd.merge(m, grp, on=bycols, how="inner")
    return m

In [4]:
from bson.son import SON # needed to ensure dictionary is ordered (python default is not)
import hashlib

def hash_feats(fts):
    vals = fts.values
    joined = "|".join(map(str,vals))
    return hashlib.sha224(joined).hexdigest()

def get_df_sorted_by_f1score(collection, params=None, filter_cols=True):
    if not params:
        params = []
    if type(params) == str:
        params = params.split(",")
    
    project = {
            "weighted_f1_score":"$WEIGHTED_MEAN_CONCEPT_CODES.f1_score",
            "micro_f1_score":  "$MICRO_F1.f1_score",
            "micro_recall":    "$MICRO_F1.recall",
            "micro_precision": "$MICRO_F1.precision",
    
    # PARAMETERS            
            "window_size":    "$parameters.window_size",
            "feats":          "$parameters.extractors",
            "count": {        "$size" : "$parameters.extractors" },
            "asof" :          "$asof",
            "_id":1
    }
    
    # No count for HMM
    if "_hmm" in collection.lower():
        del project["count"]
    
    for param in params:
        project[param] = "$parameters." + param

    feats_pipeline = [{
        "$project": project
    },
    {
        "$match":{
            "micro_f1_score": { "$exists" : True }        
        }
    },
    {
        "$sort":{
            "micro_f1_score": -1
        }
    },
    ]
    
    rows = [row for row in db[collection].aggregate(feats_pipeline)]
    df = pd.DataFrame(rows).sort_values("micro_f1_score", ascending=False)
    if params:
        df["hs_params"] = df[params].apply(hash_feats, axis=1)
        
    if filter_cols:
        cols = ["micro_f1_score", "micro_recall" ,"micro_precision" ] + params
        return df[cols]
    return df

In [9]:
def get_window_classifier_results(prefix):
    collections = "WINDOW_CLASSIFIER_BR,WINDOW_CLASSIFIER_LBL_POWERSET_MULTICLASS,WINDOW_CLASSIFIER_MOST_COMMON_TAG_MULTICLASS".split(",")
    dfs = []
    for c in collections:
        col = prefix + c
        print col
        df = dict(get_df_sorted_by_f1score(col).iloc[0,:])
        df["Collection_" + prefix[:-1]] = col.replace(prefix,"")
        dfs.append(df)
    return pd.DataFrame(dfs).sort_values("micro_f1_score", ascending=False)

# Which Problem Transformation Method Was Best?

## Coral Bleaching

In [10]:
df = get_window_classifier_results("CB_TAGGING_VD_")
df["Collection_CB_TAGGING_VD,micro_f1_score,micro_recall,micro_precision".split(",")]

CB_TAGGING_VD_WINDOW_CLASSIFIER_BR
CB_TAGGING_VD_WINDOW_CLASSIFIER_LBL_POWERSET_MULTICLASS
CB_TAGGING_VD_WINDOW_CLASSIFIER_MOST_COMMON_TAG_MULTICLASS


,Collection_CB_TAGGING_VD,micro_f1_score,micro_recall,micro_precision
2,WINDOW_CLASSIFIER_MOST_COMMON_TAG_MULTICLASS,0.831661,0.786313,0.882559
1,WINDOW_CLASSIFIER_LBL_POWERSET_MULTICLASS,0.831584,0.786103,0.882651
0,WINDOW_CLASSIFIER_BR,0.824667,0.770799,0.886629


## Skin Cancer

In [11]:
df = get_window_classifier_results("SC_TAGGING_VD_")
df["Collection_SC_TAGGING_VD,micro_f1_score,micro_recall,micro_precision".split(",")]

SC_TAGGING_VD_WINDOW_CLASSIFIER_BR
SC_TAGGING_VD_WINDOW_CLASSIFIER_LBL_POWERSET_MULTICLASS
SC_TAGGING_VD_WINDOW_CLASSIFIER_MOST_COMMON_TAG_MULTICLASS


,Collection_SC_TAGGING_VD,micro_f1_score,micro_recall,micro_precision
1,WINDOW_CLASSIFIER_LBL_POWERSET_MULTICLASS,0.808713,0.774873,0.845644
2,WINDOW_CLASSIFIER_MOST_COMMON_TAG_MULTICLASS,0.808713,0.774873,0.845644
0,WINDOW_CLASSIFIER_BR,0.801090,0.759236,0.847829


** Unsurprisingly in this case, as there were only two MLC labels, the score for LBL powerset and Common tag are the same **

** HOWEVER - why is the multiclass version that much better? It does OVR, and with only 2 records difference, this makes no sense to me **

# Hyper Parameter Tuning Results

In [8]:
# Rows to print - df.head
ROWS = 5

## Window Based Classifier - Hyper Parameter Tuning

### Coral Bleaching

In [10]:
params = "dual,C,penalty,fit_intercept,multi_class".split(",")
collection = "CB_TAGGING_VD_WINDOW_CLASSIFIER_LBL_POWERSET_MULTICLASS_HYPER_PARAM_TUNING"

df = get_df_sorted_by_f1score(collection, params)
df.head(ROWS)

KeyError: 'micro_f1_score'

### Skin Cancer

In [ ]:
params = "dual,C,penalty,fit_intercept,multi_class".split(",")
collection = "SC_TAGGING_VD_WINDOW_CLASSIFIER_LBL_POWERSET_MULTICLASS_HYPER_PARAM_TUNING"

df = get_df_sorted_by_f1score(collection, params)
df.head(ROWS)

## CRF Performance - Hyper Parameter Tuning 

### Coral Bleaching

In [ ]:
cols = "micro_f1_score,micro_precision,micro_recall,feature_possible_states,feature_possible_transitions,c2".split(",")
df = get_df_sorted_by_f1score("CB_TAGGING_VD_CRF_LBL_POWERSET_HYPERPARAM_OPT",
                         "feature_possible_states,feature_possible_transitions,c2".split(","))
df[cols].head(ROWS)

### Skin Cancer

In [ ]:
cols = "micro_f1_score,micro_precision,micro_recall,feature_possible_states,feature_possible_transitions,c2".split(",")
df = get_df_sorted_by_f1score("SC_TAGGING_VD_CRF_LBL_POWERSET_HYPERPARAM_OPT",
                         "feature_possible_states,feature_possible_transitions,c2".split(","))
df[cols].head(ROWS)

## HMM - Hyper Parameter Tuning (Features in this case)

### Coral Bleaching

In [ ]:
params = "extractors".split(",")
collection = "CB_TAGGING_VD_HMM_BR"

df = get_df_sorted_by_f1score(collection, params)
df.head(ROWS)

In [ ]:
params = "extractors".split(",")
collection = "CB_TAGGING_VD_HMM_LBL_POWERSET"

df = get_df_sorted_by_f1score(collection, params)
df.head(ROWS)

# <span style="color:red">_NEED TO DO MOST COMMON TAG_!!!!<span>

### Skin Cancer

In [ ]:
params = "extractors".split(",")
collection = "SC_TAGGING_VD_HMM_BR"

df = get_df_sorted_by_f1score(collection, params)
df.head(ROWS)

## Average Perceptron - Hyper Parameter Tuning Results

### Coral Bleaching

In [ ]:
model = "AVG_PERCEPTRON_MULTICLASS"
df = get_df_sorted_by_f1score("CB_TAGGING_VD_" + model, 
                              "prev_tag_sharing,num_iterations,tag_history,combo_freq_threshold")
df.head(ROWS)["micro_f1_score,micro_precision,micro_recall,prev_tag_sharing,num_iterations,tag_history,combo_freq_threshold".split(",")]

### Skin Cancer

In [ ]:
model = "AVG_PERCEPTRON_MULTICLASS"
df = get_df_sorted_by_f1score("SC_TAGGING_VD_" + model, 
                              "prev_tag_sharing,num_iterations,tag_history,combo_freq_threshold")
df.head(ROWS)["micro_f1_score,micro_precision,micro_recall,prev_tag_sharing,num_iterations,tag_history,combo_freq_threshold".split(",")]

# Latest

In [ ]:
params = "dual,C,penalty,fit_intercept,multi_class".split(",")
collection = "CB_TAGGING_VD_WINDOW_CLASSIFIER_MOST_COMMON_TAG_MULTICLASS_HYPER_PARAM_TUNING"

df = get_df_sorted_by_f1score(collection, params)
#df.head(ROWS)
df.head()

In [ ]:
params = "dual,C,penalty,fit_intercept,multi_class".split(",")
collection = "CB_TAGGING_VD_WINDOW_CLASSIFIER_LBL_POWERSET_MULTICLASS_HYPER_PARAM_TUNING"

df = get_df_sorted_by_f1score(collection, params)
df.head(ROWS)
#df.sort_values(["dual","penalty","C"])